# Optimal shopping problem

I need to get some goods to met a list of requirements.

There are a ton of goods that can partially satisfy one or more of my requirements. I cannot get a fraction of a good.

How many goods do I get? Which ones? I need to solve an integer optimization problem.

$$
\begin{array}{lll} 
    \mbox{Minimize} & \sum_{i=1}^{N} c_ix_i & \to \text{Minimize cost of goods}\\
    \mbox{Subject to} & \sum_{i=1}^{N} x_i q_{ij} \geqslant m_j \quad \forall j & \to \text{Fulfilling my requirements} \\
    & x_i \geqslant 0, x_i \in \mathbb{N} \quad \forall i \\
\end{array}
$$

Where:

$
\begin{array}{ll} 
    \mbox{$x_i$}    : & \text{Quantity of good } i \text{ to be obtained} \\
    \mbox{$c_i$}    : & \text{Cost of good } i \\
    \mbox{$m_j$}    : & \text{My total requirement of type} j \\
    \mbox{$q_{ij}$} : & \text{Quantity of requirement } j \text{ fulfilled by each good } i \text{ obtained}    
\end{array}
$

If the total cost is not of our interest we can simple let $c_i = 1, \forall i$, and the problem will minimize the total quantity of goods needed to fullfil the requirements.

## Random parameters generation

In [24]:
import cvxpy as cvx
import numpy as np
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

i = 10   # Type of goods available
j = 5    # Type of needs to be fulfilled

# Random generation of C, Q and M
C = np.random.randint(1,10,i)         # Cost of each good
Q = np.random.choice([0,1,2], (j,i))  # Quantity of each need fulfillef for each good
m = np.random.randint(1,100,(j))      # Requirements of each need

print("Number of goods: \ni =", i, "\n")
print("Number of requirements: \nj =", j, "\n")
print("Cost of each good: \nC =", C, "\n")
print("Quantity of each requirement: \nm =", m, "\n")
print("Quantity of each requirement fulfilled by each good: \nQ =")
print(*Q, sep=' \n')

Number of goods: 
i = 10 

Number of requirements: 
j = 5 

Cost of each good: 
C = [8 7 4 7 9 5 9 3 4 7] 

Quantity of each requirement: 
m = [75 94 86 43 68] 

Quantity of each requirement fulfilled by each good: 
Q =
[0 0 2 1 2 1 2 1 0 2] 
[0 2 0 1 1 1 1 2 2 1] 
[1 1 2 1 1 2 0 1 1 0] 
[2 1 0 1 2 1 0 0 1 0] 
[2 0 2 2 2 1 1 1 1 2]


## Problem construction and solution using CVXPY

We will use [CVXPY](https://www.cvxpy.org/) to construct and solve the optimization problem.

To make it simple we use vector notation:

$
\begin{array}{lll} 
    \mbox{Minimize} & \mathbf{Cx} \\
    \mbox{Subject to} & \mathbf{Qx} \geqslant \mathbf{m} \\
    \mbox{} & \mathbf{x} \geqslant 0, \mathbf{x} \in \mathbb{N}
\end{array}
$

In [25]:
# Construct the problem
x = cvx.Variable(i, integer=True)

objective = cvx.Minimize(C*x)
constraints = [Q*x >= m, x >= 0]

prob = cvx.Problem(objective, constraints)

In [26]:
# Solve the problem
sol = prob.solve()

if(prob.status == 'optimal'):
    print("SOLUTION FOUND! \nObjective function value =", sol, prob.status)
else:
    print("ERROR: ", prob.status)

SOLUTION FOUND! 
Objective function value = 300.0000001467965 optimal


## Optimal solution verification

In [28]:
x_opt = x.value[np.newaxis].T
print("Optimal goods to get: \n x_optimal =", *x_opt.astype(int))
#print("Optimal goods to get: \n x_optimal =", *x_opt)

Optimal goods to get: 
 x_optimal = [0] [0] [1] [0] [8] [25] [0] [29] [0] [0]


The fulfilled requirements always must be greater or equal to the original requirements:

In [29]:
print("Original requirements : \n m =", m, "\n")
print("Fulfilled requirements: \n m_optimal =", *Q.dot(x_opt).T)

Original requirements : 
 m = [75 94 86 43 68] 

Fulfilled requirements: 
 m_optimal = [75. 94. 91. 43. 75.]


In [30]:
print("Total number of goods to get =", x_opt.astype(int).sum())
print("Total cost =", C.dot(x.value))

Total number of goods to get = 63
Total cost = 300.0000001467964


If C = [1 1 1 ...], the total number of goods to get must be equal to the total cost of goods.